In [1]:
################################################################
#This file will summarize data from the "Heros of Pamoli" game#
###############################################################

In [2]:
#import dependencies
import pandas as pd
import numpy as np

In [4]:
#read the master data file and create a dataframe
masterFile = "Resources/purchase_data.csv"
master_df = pd.read_csv(masterFile)
master_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [6]:
#create the age group category for later and add to dataframe, completing my master dataframe
bins=[0,9,14,19,24,29,34,39,400]
labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
master_df["Age Group"] = pd.cut(master_df["Age"], bins, labels=labels, include_lowest=True)

In [7]:
#Create a dataframe to display total number of players
unique_players=master_df["SN"].unique()
total_players=np.count_nonzero(unique_players)
total_players_df =pd.DataFrame([
    {"Total Players":total_players}])
total_players_df

,Total Players
0,576


In [8]:
#Create a dataframe to display basic info about purchases
unique_items=master_df["Item Name"].unique()
total_unique_items=np.count_nonzero(unique_items)
average_price=master_df["Price"].mean()
total_purchases=master_df["Purchase ID"].count()
total_revenue=master_df["Price"].sum()

basic_info = pd.DataFrame([
        {"Number of Unique Items": total_unique_items,
         "Average Price":average_price,
         "Number of Purchases": total_purchases,
         "Total Revenue": total_revenue}
    ])
basic_info

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [13]:
#create a dataframe to display the gender breakdown of players
unique_sn_df = master_df.drop_duplicates(subset='SN', keep="first")
male_players = unique_sn_df.loc[master_df["Gender"] == "Male", "Gender"]
female_players = unique_sn_df.loc[master_df["Gender"] == "Female", "Gender"]
other_players = unique_sn_df.loc[master_df["Gender"] == "Other / Non-Disclosed", "Gender"]

playerCount_df = pd.DataFrame({ 
    "Gender": ["Female", "Male", "Other"],
    "Total Count": [female_players.count(), male_players.count(), other_players.count()],
   })

percent_gender = playerCount_df["Total Count"]/total_players*100
playerCount_df["Percentage of Players"] = percent_gender

playerCount_df

,Gender,Total Count,Percentage of Players
0,Female,81,14.062500
1,Male,484,84.027778
2,Other,11,1.909722


In [14]:
#Create a table of summary statistics concerning Male and Female purchases

# Use GroupBy to get most of our Gender data
grouped_gender_df = master_df.groupby(['Gender'])

#Get our data functions from grouped gender data
gender_priceMean=grouped_gender_df["Price"].mean()
gender_purchaseCount=grouped_gender_df["Purchase ID"].count()
gender_totalvalue=grouped_gender_df["Price"].sum()

#### Get the average total purchase per person. I know there's an easier way to do this, but I haven't found it yet.
#Get female average spent per person
female_df=master_df.loc[master_df["Gender"]=="Female", :] #dataset of only females
grouped_gender_sn_df = female_df.groupby(['SN'])
F_total_purchase_per=grouped_gender_sn_df["Price"].sum()
F_total_purchase_per.mean()

#Get male average spent per person
male_df=master_df.loc[master_df["Gender"]=="Male", :] #dataset of only Males
grouped_gender_sn_df = male_df.groupby(['SN'])
M_total_purchase_per=grouped_gender_sn_df["Price"].sum()
M_total_purchase_per.mean()

#Get other gender average spent per person
other_df=master_df.loc[master_df["Gender"]=="Other / Non-Disclosed", :] #dataset of only Others
grouped_gender_sn_df = other_df.groupby(['SN'])
O_total_purchase_per=grouped_gender_sn_df["Price"].sum()
O_total_purchase_per.mean()

#Put these in a list
avg_total_list=[F_total_purchase_per.mean(), M_total_purchase_per.mean(),O_total_purchase_per.mean()]
####

#Put everything in a dataframe
gender_stats_df = pd.DataFrame({"Purchase Count": gender_purchaseCount,
                                "Average Purchase Price": gender_priceMean,
                                "Total Purchase Value": gender_totalvalue,
                               "Average Total Purchase per Person": avg_total_list})

gender_stats_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [15]:
#Create a table of summary statistics for players of different age ranges

master_df["Age Group"] = pd.cut(master_df["Age"], bins, labels=labels, include_lowest=True)
age_group_df=unique_sn_df.groupby("Age Group")
age_count=age_group_df["Purchase ID"].count()
age_percent =age_count/total_players
age_df = pd.DataFrame({"Total Count":age_count,
                      "Percentage of Players":age_percent})
age_df

,Total Count,Percentage of Players
Age Group,,
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833
